In [1]:
import os
import cv2
import sounddevice as sd
import numpy as np
import tkinter as tk
from scipy.io.wavfile import write, read
import speech_recognition as sr
import language_tool_python
import spacy
import spacy
import tensorflow as tf
from tensorflow import keras
import librosa



class VideoRecorderApp:
    def __init__(self, master, question):
        self.master = master
        self.question = question
        self.master.title("Video Recorder")
        self.master.geometry("800x600")

        self.label = tk.Label(self.master, text=f"Question: {self.question}")
        self.label.pack()

        self.start_button = tk.Button(self.master, text="Start Recording", command=self.start_recording)
        self.start_button.pack()

        self.stop_button = tk.Button(self.master, text="Stop Recording", command=self.stop_recording, state=tk.DISABLED)
        self.stop_button.pack()

        self.cap = None
        self.out = None
        self.audio_data = []

    def start_recording(self):
        self.start_button.config(state=tk.DISABLED)
        self.stop_button.config(state=tk.NORMAL)

        self.cap = cv2.VideoCapture(0)
        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        self.out = cv2.VideoWriter('D:\\AI_interview\\recorded\\answer.avi', fourcc, 20.0, (640, 480))
        

        def callback(indata, frames, time, status):
            if status:
                print(status, flush=True)
            self.audio_data.append(indata.copy())

        # Flag to check if 'e' key is pressed
        stop_recording = False

        # Start recording audio using the callback function
        with sd.InputStream(callback=callback):
            while True:
                ret, frame = self.cap.read()
                cv2.imshow('Recording', frame)
                self.out.write(frame)

                # Check if 'e' key is pressed
                key = cv2.waitKey(1) & 0xFF
                if key == ord('e'):
                    stop_recording = True
                    break

        # Stop audio recording when 'e' key is pressed
        sd.stop()

        # Check if 'e' key was pressed during video recording
        if stop_recording:

            # Release video resources
            self.cap.release()
            self.out.release()

            audio_array = np.concatenate(self.audio_data, axis=0)
            audio_path = os.path.join("D:\\AI_interview\\recorded", "answer.wav")
            print("Saving audio file to:", audio_path)  # Print the path
            write(audio_path, 44100, audio_array.astype(np.int16))



            # Close OpenCV windows
            cv2.destroyAllWindows()

        else:
            # If 'e' key wasn't pressed, release video resources and show a message
            self.cap.release()
            self.out.release()
            cv2.destroyAllWindows()
            print("Recording stopped without saving audio.")

    def stop_recording(self):
        self.start_button.config(state=tk.NORMAL)
        self.stop_button.config(state=tk.DISABLED)

        # Release video resources
        self.cap.release()
        self.out.release()

        # Concatenate audio data and save as WAV file to specified path
        audio_array = np.concatenate(self.audio_data, axis=0)
        # Normalize the audio data before saving
        normalized_audio = (audio_array / np.max(np.abs(audio_array)) * 32767).astype(np.int16)

        # Concatenate audio data and save as WAV file to specified path
        audio_path = os.path.join("D:\\AI_interview\\recorded", "answer.wav")

        print("Saving audio file to:", audio_path)  # Print the path
        write(audio_path, 44100, normalized_audio)


        cv2.destroyAllWindows()


video_path = 'D:\\AI_interview\\recorded\\answer.avi'
audio_path = 'D:\\AI_interview\\recorded\\answer.wav'



def ask_question():
    
    print("Choose a question number between 1 and 6:")
    question_number = int(input())
    questions = [
        "What is supervised learning and give some examples?",
        "What is unsupervised learning and give some examples?",
        "What is the definition of reinforcement learning?",
        "What is overfitting and how can we avoid or solve it?",
        "What is cross-validation and give an example?",
        "What is bias-variance tradeoff?"
    ]

    question = questions[question_number - 1]

    root = tk.Tk()
    app = VideoRecorderApp(root, question)
    root.mainloop()

    

    def extract_frames(video_path, output_path):
        # Open the video file
        cap = cv2.VideoCapture(video_path)

        # Get video properties
        fps = cap.get(cv2.CAP_PROP_FPS)
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        duration = total_frames / fps

        # Calculate the frame interval to get n/2 frames
        frame_interval = int((total_frames * 4) /  duration)

        # Create output directory if it doesn't exist
        if not os.path.exists(output_path):
            os.makedirs(output_path)

        # Loop through the video and extract frames
        current_frame = 0
        while True:
            ret, frame = cap.read()
        
            if not ret:
                break  # Break the loop if no more frames are available

            # Save the frame if it's at the interval
            if current_frame % frame_interval == 0:
                frame_filename = f"{output_path}/frame_{current_frame}.jpg"
                cv2.imwrite(frame_filename, frame)

            current_frame += 1

        # Release the video capture object
        cap.release()




    def predict_emotion(frame_path):
        img = cv2.imread(frame_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)  # Convert to grayscale if needed
        img = cv2.resize(img, (48, 48))  # Resize to match the input size of the model
        img = img / 255.0  # Normalize pixel values
    
        img = img.reshape((1,) + img.shape + (1,))
    
        # Make prediction
        emotion_probs = emotion_model.predict(img)
    
        # Get the emotion with the highest probability
        predicted_emotion_video = emotion_classes_video[np.argmax(emotion_probs)]
    
        return predicted_emotion_video





    def calculate_emotion_score(emotion):
        emotion_scores_video = {'angry': 40, 'fear': 40, 'happy': 100, 'neutral': 65, 'sad': 50, 'surprise': 55}
    
        return emotion_scores_video.get(emotion, 0)



        
        
    
        # Load the pre-trained emotion detection model
    video_model_path = r"D:\AI_interview\facerec\emotion_detector_final_model.h5"
    emotion_model = tf.keras.models.load_model(video_model_path)

    # Define the emotion classes
    emotion_classes_video = ['angry', 'fear', 'happy', 'neutral', 'sad', 'surprise']

    # Specify video path and output directory
    output_path = r"D:\AI_interview\extracted"

    # Extract frames from the video
    extract_frames(video_path, output_path)

    # Check the content of the output directory
    print("Frames extracted:")
    print(os.listdir(output_path))

    # Predict emotions for each frame and calculate average emotion score
    emotions_scores_video = []

    for frame_file in os.listdir(output_path):
        frame_path = os.path.join(output_path, frame_file)
        predicted_emotion_video = predict_emotion(frame_path)
        video_emotion_score = calculate_emotion_score(predicted_emotion_video)
        emotions_scores_video.append(video_emotion_score)

    # Check the content of emotion_scores
    print("Emotion Scores_video:")
    print(emotions_scores_video)

    # Check if emotion_scores is empty
    if not emotions_scores_video:
        print("No frames were processed. Please check the frame extraction process.")
    else:
        # Calculate average emotion score
        average_emotion_score = np.nanmean(emotions_scores_video)
        print("Average Emotion Score:", int(average_emotion_score))






    # Load the trained model
    audio_model_path = "D:/AI_interview/audio_confidence/AI_audio2.h5"
    audio_model = tf.keras.models.load_model(audio_model_path)

    # Define emotion classes
    emotion_classes_audio = ["happy", "angry", "sad", "neutral", "calm", "fear", "disgust", "surprise"]


    # Load the audio file and extract features
    def extract_features(audio_path):
        audio, sr = librosa.load(audio_path, sr=None)
        mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=13)

        # Pad or truncate the features to match the expected input shape (162 frames)
        if mfccs.shape[1] < 162:
            mfccs = np.pad(mfccs, ((0, 0), (0, 162 - mfccs.shape[1])))
        else:
            mfccs = mfccs[:, :162]
    
        return mfccs

    # Preprocess the audio for model prediction
    input_data = extract_features(audio_path)
    input_data = np.expand_dims(input_data, axis=-1)  # Add channel dimension

    # Make predictions
    predictions_audio = audio_model.predict(input_data)

    # Map predictions to emotion classes
    predicted_class_index = np.argmax(predictions_audio)
    predicted_emotion_audio = emotion_classes_audio[predicted_class_index]

    # Score the prediction
    emotion_score_audio = 0
    confidence_score = 0

    if predicted_emotion_audio in ["happy", "calm", "neutral"]:
        emotion_score_audio = 100
        confidence_score = 100
    elif predicted_emotion_audio in ["sad", "fear", "surprise"]:
        emotion_score_audio = 60
        confidence_score = 50
    else:  # ["disgust", "angry"]
        emotion_score_audio = 40
        confidence_score = 40

    # Print the results
    print(f"Predicted Emotion: {predicted_emotion_audio}")
    print(f"Emotion Score: {emotion_score_audio}")
    print(f"Confidence Score: {confidence_score}")
    
    
    
    
    supervised_learning = "Supervised learning is a type of machine learning where the algorithm is trained on a labeled dataset, meaning that the input data is paired with corresponding output labels. The goal of supervised learning is to learn a mapping function from the input variables to the output variables, based on the labeled examples provided during training. In other words, the algorithm learns to make predictions or decisions by generalizing patterns from the labeled training data.Examples of supervised learning include image classification, spam email detection, handwriting recognition, speech recognition, predicting stock prices, language translation, medical diagnosis, customer churn prediction, autonomous vehicles, and credit scoring."
    unsupervised_learning = "Unsupervised learning is a machine learning paradigm where the algorithm is trained on unlabeled data, and the objective is to discover inherent patterns or structures within the data without explicit guidance. In unsupervised learning, the algorithm explores the data's inherent structure, often through clustering or dimensionality reduction techniques. Examples of unsupervised learning include clustering similar documents in a large corpus, anomaly detection to identify unusual patterns, dimensionality reduction for feature extraction, topic modeling to discover themes in text data, and generative modeling for creating new data instances based on the learned patterns"
    reinforcement_learning = "Reinforcement learning is a type of machine learning where an agent learns to make decisions by interacting with an environment. The agent receives feedback in the form of rewards or penalties based on its actions. The goal is for the agent to learn a policy that maximizes the cumulative reward over time."
    overfitting = "Overfitting occurs when a machine learning model learns the training data too well, including its noise and outliers, leading to poor generalization on new, unseen data. It often results from models being too complex relative to the amount of training data. To address overfitting, techniques such as regularization, reducing model complexity, increasing the size of the training dataset, and using cross-validation to evaluate model performance on different subsets of data can be employed"
    cross_validation = "Cross-validation is a technique used to assess a model's performance by partitioning the dataset into multiple subsets. The model is trained on some subsets and tested on others, allowing for a more robust evaluation. Common methods include k-fold cross-validation, where the data is divided into k subsets, and the model is trained and tested k times, each time using a different subset for testing."
    bias_variance_tradeoff = "The bias-variance tradeoff is a fundamental concept in machine learning that deals with finding the right level of model complexity. High bias (underfitting) occurs when a model is too simple and cannot capture the underlying patterns in the data. High variance (overfitting) occurs when a model is too complex and fits the training data too closely, failing to generalize well to new data. Balancing bias and variance is crucial for building models that perform well on diverse datasets. Regularization techniques and model selection are commonly used to manage the bias-variance tradeoff"


    r = sr.Recognizer()


    def speech_to_text(audio_file):
      """
      Opens and listens to an audio file and translates it to text
      Args: audio file
      Returns: text of transcribed audio file
      """
      with sr.AudioFile(audio_file) as source:
        audio_text = r.listen(source)
        try:
            text = r.recognize_google(audio_text)
            print('Converting audio transcripts into text ...')
            # print(text)
        except:
             print('Sorry.. run again...')

        return text





    def extract_keywords(text):
        nlp = spacy.load("en_core_web_sm")
        doc = nlp(text)
        keywords = [token.text for token in doc if token.is_alpha]
        return keywords



    def check_semantic(text):
        nlp = spacy.load("en_core_web_sm")
        doc = nlp(text)

        # Example: Check for specific semantic elements like named entities
        entities = [ent.text for ent in doc.ents]

        # You can also use dependency parsing to analyze sentence structure
        # For example, check if certain words are connected in a specific way
        root_verb = [token.text for token in doc if token.dep_ == 'ROOT' and token.pos_ == 'VERB']

        # Example custom scoring function: Assign a score based on the number of entities
        # and the presence of a root verb
        semantic_score = calculate_semantic_score(entities, root_verb)

        return semantic_score



    def calculate_semantic_score(entities, root_verb):
        # Example: Assign a higher score if entities and a root verb are present
        score = 0
        if entities:
            score += 0.5
        if root_verb:
            score += 0.5
        return score
    


    def check_syntax(text):
        tool = language_tool_python.LanguageTool('en-US')
        matches = tool.check(text)

        # Count the number of matches
        num_matches = len(matches)

        # Assign a score based on the number of matches
        # You can customize this scoring logic based on your requirements
        score = (1 - (num_matches*0.035))  # For example, inverse of the number of matches

        return score, matches



    def score_answer(chosen_number, keywords):
        correct_answers = [
            "Supervised learning is a type of machine learning where the algorithm is trained on a labeled dataset, meaning that the input data is paired with corresponding output labels. The goal of supervised learning is to learn a mapping function from the input variables to the output variables, based on the labeled examples provided during training. In other words, the algorithm learns to make predictions or decisions by generalizing patterns from the labeled training data.Examples of supervised learning include image classification, spam email detection, handwriting recognition, speech recognition, predicting stock prices, language translation, medical diagnosis, customer churn prediction, autonomous vehicles, and credit scoring.",
             "Unsupervised learning is a machine learning paradigm where the algorithm is trained on unlabeled data, and the objective is to discover inherent patterns or structures within the data without explicit guidance. In unsupervised learning, the algorithm explores the data's inherent structure, often through clustering or dimensionality reduction techniques. Examples of unsupervised learning include clustering similar documents in a large corpus, anomaly detection to identify unusual patterns, dimensionality reduction for feature extraction, topic modeling to discover themes in text data, and generative modeling for creating new data instances based on the learned patterns",
            "Reinforcement learning is a type of machine learning where an agent learns to make decisions by interacting with an environment. The agent receives feedback in the form of rewards or penalties based on its actions. The goal is for the agent to learn a policy that maximizes the cumulative reward over time.",
            "Overfitting occurs when a machine learning model learns the training data too well, including its noise and outliers, leading to poor generalization on new, unseen data. It often results from models being too complex relative to the amount of training data. To address overfitting, techniques such as regularization, reducing model complexity, increasing the size of the training dataset, and using cross-validation to evaluate model performance on different subsets of data can be employed",
            "Cross-validation is a technique used to assess a model's performance by partitioning the dataset into multiple subsets. The model is trained on some subsets and tested on others, allowing for a more robust evaluation. Common methods include k-fold cross-validation, where the data is divided into k subsets, and the model is trained and tested k times, each time using a different subset for testing.",
            "The bias-variance tradeoff is a fundamental concept in machine learning that deals with finding the right level of model complexity. High bias (underfitting) occurs when a model is too simple and cannot capture the underlying patterns in the data. High variance (overfitting) occurs when a model is too complex and fits the training data too closely, failing to generalize well to new data. Balancing bias and variance is crucial for building models that perform well on diverse datasets. Regularization techniques and model selection are commonly used to manage the bias-variance tradeoff"
        ]

        correct_answer = correct_answers[chosen_number - 1]  # Adjust for 0-based index

        keyword_match = set(keywords) & set(correct_answer.split())
        keyword_score = len(keyword_match) / len(correct_answer.split()) * 100
        return keyword_score


    def calculate_scores(semantic, syntax, answer):
        total_score = 0.1 * semantic + 0.1 * syntax + 0.8 * answer
        return total_score



    def evaluate_student_answer(audio_path, chosen_number):
        # Assuming correct_answer_keywords is a list of keywords related to the correct answer
        correct_answer_keywords = get_correct_answer_keywords(chosen_number)
        # Rest of the code remains the same
        text = speech_to_text(audio_path)
        keywords = extract_keywords(text)
        syntax_matches = check_syntax(text)
        semantic_score = check_semantic(text)
        answer_score = score_answer(chosen_number, keywords)  # Fix here
        syntax_score = 100 - (len(syntax_matches) * 3.5)  # Adjust as needed
        final_score = calculate_scores(semantic_score, syntax_score, answer_score)

        return final_score

    def get_correct_answer_keywords(chosen_number):
        # Assuming chosen_number is between 1 and 6
        chosen_index = chosen_number - 1
        correct_answers = [
            "Supervised learning is a type of machine learning where the algorithm is trained on a labeled dataset, meaning that the input data is paired with corresponding output labels. The goal of supervised learning is to learn a mapping function from the input variables to the output variables, based on the labeled examples provided during training. In other words, the algorithm learns to make predictions or decisions by generalizing patterns from the labeled training data.Examples of supervised learning include image classification, spam email detection, handwriting recognition, speech recognition, predicting stock prices, language translation, medical diagnosis, customer churn prediction, autonomous vehicles, and credit scoring.",
            "Unsupervised learning is a machine learning paradigm where the algorithm is trained on unlabeled data, and the objective is to discover inherent patterns or structures within the data without explicit guidance. In unsupervised learning, the algorithm explores the data's inherent structure, often through clustering or dimensionality reduction techniques. Examples of unsupervised learning include clustering similar documents in a large corpus, anomaly detection to identify unusual patterns, dimensionality reduction for feature extraction, topic modeling to discover themes in text data, and generative modeling for creating new data instances based on the learned patterns",
            "Reinforcement learning is a type of machine learning where an agent learns to make decisions by interacting with an environment. The agent receives feedback in the form of rewards or penalties based on its actions. The goal is for the agent to learn a policy that maximizes the cumulative reward over time.",
            "Overfitting occurs when a machine learning model learns the training data too well, including its noise and outliers, leading to poor generalization on new, unseen data. It often results from models being too complex relative to the amount of training data. To address overfitting, techniques such as regularization, reducing model complexity, increasing the size of the training dataset, and using cross-validation to evaluate model performance on different subsets of data can be employed",
            "Cross-validation is a technique used to assess a model's performance by partitioning the dataset into multiple subsets. The model is trained on some subsets and tested on others, allowing for a more robust evaluation. Common methods include k-fold cross-validation, where the data is divided into k subsets, and the model is trained and tested k times, each time using a different subset for testing.",
            "The bias-variance tradeoff is a fundamental concept in machine learning that deals with finding the right level of model complexity. High bias (underfitting) occurs when a model is too simple and cannot capture the underlying patterns in the data. High variance (overfitting) occurs when a model is too complex and fits the training data too closely, failing to generalize well to new data. Balancing bias and variance is crucial for building models that perform well on diverse datasets. Regularization techniques and model selection are commonly used to manage the bias-variance tradeoff"
        ]

        return extract_keywords(correct_answers[chosen_index])

    # Example usage
    chosen_number = question_number  # Assuming the student picked 1
    paths = audio_path
    score = evaluate_student_answer(paths, chosen_number)
    print(f"Final Score: {score}")

    def final_mark(a,b,c,d):
        final_marks = (0.2 * a) + (0.2 * b) + (0.1 * c) + (0.5 * d)
        return final_marks

    total = final_mark(average_emotion_score,emotion_score_audio,confidence_score,score)
    print("final marks of the student is :",total)
    

if __name__ == "__main__":
    ask_question()

ModuleNotFoundError: No module named 'cv2'